<a href="https://colab.research.google.com/github/biswajitmohanty/genai/blob/main/Simple_RNN_Predict_7_days_Stock_Price_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go


def fetch_stock_data(tickers, period="5y"):
    stock_data = {}
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)
        stock_data[ticker] = df[["Close"]]
    return stock_data

In [ ]:
def create_sequences(data, time_steps=60, future_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps - future_steps + 1):
        X.append(data[i:i+time_steps])
        y.append(data[i+time_steps:i+time_steps+future_steps])
    return np.array(X), np.array(y)

In [ ]:
def build_rnn_model(time_steps, future_steps):
    model = Sequential([
        SimpleRNN(50, activation='relu', return_sequences=True, input_shape=(time_steps, 1)),
        Dropout(0.2),
        SimpleRNN(50, activation='relu'),
        Dropout(0.2),
        Dense(future_steps)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
def train_and_predict(tickers, time_steps=60, future_steps=1, epochs=10):
    stock_data = fetch_stock_data(tickers)
    predictions = {}

    for ticker, df in stock_data.items():
        scaler = MinMaxScaler()
        df_scaled = scaler.fit_transform(df)

        X, y = create_sequences(df_scaled, time_steps, future_steps)

        model = build_rnn_model(time_steps, future_steps)
        model.fit(X, y, epochs=epochs, batch_size=32, verbose=1)

        last_sequence = df_scaled[-time_steps:].reshape(1, time_steps, 1)
        prediction = model.predict(last_sequence)
        predicted_prices = scaler.inverse_transform(prediction)
        predictions[ticker] = predicted_prices

        # Plot actual vs predicted
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index[-100:], y=df["Close"].iloc[-100:], mode='lines', name="Actual Price"))

        future_dates = [df.index[-1] + pd.Timedelta(days=i+1) for i in range(future_steps)]
        fig.add_trace(go.Scatter(x=future_dates, y=predicted_prices.flatten(), mode='lines+markers', name="Predicted Prices"))

        fig.update_layout(title=f"{ticker} Stock Price Prediction for Next {future_steps} Days",
                          xaxis_title="Date", yaxis_title="Price", template="plotly_dark")
        fig.show()

    return predictions

In [ ]:
time_steps_options = [30, 60, 90]
tickers = ["MSFT", "AAPL", "GOOGL", "TSLA"]
predictions = {}

for time_steps in time_steps_options:
    predictions[time_steps] = train_and_predict(tickers, time_steps, future_steps=7)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



39/39 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.1822
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0284
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0217
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0177
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0152
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0126
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0122
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0102
Epoch 9/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0094
Epoch 10/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0092
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.2072
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0271
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0198
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0149
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0136
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 0.0122
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0103
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0101
Epoch 9/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0079
Epoch 10/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0085
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.1218
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - loss: 0.0184
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0144
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0119
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0097
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0100
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0089
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0081
Epoch 9/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0076
Epoch 10/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0071
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step


In [ ]:
for time_steps, stock_preds in predictions.items():
    for ticker, pred in stock_preds.items():
        pd.DataFrame(pred, columns=[f"Day {i+1}" for i in range(7)]).to_csv(f"{ticker}_predictions_{time_steps}.csv", index=False)
